# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [10]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
"""
Your Code Here
"""
column='Embarked'
import statistics
mean_df = df.groupby([column])['Fare'].mean().reset_index()
print (mean_df.head())
mode_df = df.groupby([column])['Fare'].apply(lambda x: x.mode()[0]).reset_index()
#mode_df = df.groupby(['Cabin'])['Fare'].mode().reset_index()
median_df = df.groupby([column])['Fare'].median().reset_index()
max_df = df.groupby([column])['Fare'].max().reset_index()
temp = pd.merge(mean_df, mode_df, on = column, how = 'left')
temp = pd.merge(temp, median_df, on = column, how = 'left')
temp = pd.merge(temp, max_df, on = column, how = 'left')
temp.columns = ('Embarked', 'mean_df', 'mode_df', 'median_df', 'max_df')
df = pd.merge(df, temp, on = column, how = 'left')
print (df.head())
#print (df[column].nunique())

  Embarked       Fare
0        C  59.954144
1        Q  13.276030
2        S  27.079812
   Pclass                                               Name     Sex   Age  \
0       3                            Braund, Mr. Owen Harris    male  22.0   
1       1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0   
2       3                             Heikkinen, Miss. Laina  female  26.0   
3       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0   
4       3                           Allen, Mr. William Henry    male  35.0   

   SibSp  Parch            Ticket     Fare Cabin Embarked    mean_df  mode_df  \
0      1      0         A/5 21171   7.2500   NaN        S  27.079812   8.0500   
1      1      0          PC 17599  71.2833   C85        C  59.954144   7.2292   
2      0      0  STON/O2. 3101282   7.9250   NaN        S  27.079812   8.0500   
3      1      0            113803  53.1000  C123        S  27.079812   8.0500   
4      0      0            373450   8.

In [12]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.describe()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'mean_df', 'mode_df', 'median_df', 'max_df']



,Pclass,Age,SibSp,Parch,Fare,mean_df,mode_df,median_df,max_df
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,2.308642,23.600640,0.523008,0.381594,32.204208,32.022390,7.848996,15.663692,294.468356
std,0.836071,17.867496,1.102743,0.806057,49.693429,14.064211,0.527790,6.954584,116.142754
min,1.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,2.000000,6.000000,0.000000,0.000000,7.910400,27.079812,7.750000,13.000000,263.000000
50%,3.000000,24.000000,0.000000,0.000000,14.454200,27.079812,8.050000,13.000000,263.000000
75%,3.000000,35.000000,1.000000,0.000000,31.000000,27.079812,8.050000,13.000000,263.000000
max,3.000000,80.000000,8.000000,6.000000,512.329200,59.954144,8.050000,29.700000,512.329200


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [13]:
# 原始特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_temp = df.drop(['mean_df', 'mode_df', 'median_df', 'max_df'], axis = 1)
print (df_temp.head())
train_num = train_Y.shape[0]
train_X = MMEncoder.fit_transform(df_temp)[:train_num]
regressor = LogisticRegression()
accuracy = cross_val_score(regressor, train_X, train_Y, cv= 5)
print (accuracy.mean(), '+-', accuracy.std())

   Pclass   Age  SibSp  Parch     Fare
0       3  22.0      1      0   7.2500
1       1  38.0      1      0  71.2833
2       3  26.0      0      0   7.9250
3       1  35.0      1      0  53.1000
4       3  35.0      0      0   8.0500
0.7038635542329971 +- 0.04592013735420995


In [14]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
df_temp = df[['mean_df', 'mode_df', 'median_df', 'max_df']]
train_X = MMEncoder.fit_transform(df_temp)[:train_num]
regressor = LogisticRegression()
accuracy = cross_val_score(regressor, train_X, train_Y, cv= 5)
print (accuracy.mean(), '+-', accuracy.std())

0.6375829096311174 +- 0.03166460065537792


In [15]:
df_temp = df
train_X = MMEncoder.fit_transform(df_temp)[:train_num]
regressor = LogisticRegression()
accuracy = cross_val_score(regressor, train_X, train_Y, cv= 5)
print (accuracy.mean(), '+-', accuracy.std())

0.7005053927832138 +- 0.04804524364702929
